In [2]:
import tensorflow as tf
import numpy as np
import skimage.transform
import matplotlib.pyplot as plt
import warnings
import tqdm
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Load graph data
loader = tf.train.import_meta_graph('saved_model/classifier.ckpt.meta')
sess = tf.Session()
loader.restore(sess,'saved_model/classifier.ckpt')
graph = tf.get_default_graph()

INFO:tensorflow:Restoring parameters from saved_model/classifier.ckpt


In [3]:
# Find relevant ops
print([op.name for op in graph.get_operations() if 'output' in op.name or 'features' in op.name])

['network_input/features', 'cnn/output']


In [4]:
# Grab them
x = graph.get_tensor_by_name('network_input/features:0')
y = graph.get_tensor_by_name('cnn/output:0')

In [5]:
test_image = skimage.transform.resize(plt.imread('train/dog.0.jpg'), (128,128,1))
res = sess.run(y, feed_dict={x: [test_image]})
answer = 'cat' if res[0][0]>res[0][1] else 'dog'
print(f'{res[0]} - {answer}')

[0.27150866 0.7284913 ] - dog


In [6]:
counter = 0
for i in tqdm.trange(1000):
    animal = 'cat' if i%2==0 else 'dog'
    test_image = skimage.transform.resize(plt.imread(f'train/{animal}.{i}.jpg'), (128,128,1))
    res = sess.run(y, feed_dict={x: [test_image]})
    answer = 'cat' if res[0][0]>res[0][1] else 'dog'
    if answer == animal:
        counter+=1
        
print(f'{counter/10}% accuracy')

100%|██████████████████████████████| 1000/1000 [00:23<00:00, 43.30it/s]


0.799% accuracy
